In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

%matplotlib inline

In [2]:
train_x = np.load('train_x.npy')
train_y = np.load('train_y.npy')

val_x = np.load('val_x.npy')
val_y = np.load('val_y.npy')

In [3]:
train_x = train_x[:37400]
train_y = train_y[:37400]

In [4]:
val_x = val_x[:4150]
val_y = val_y[:4150]

In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)

In [6]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
# print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
# print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])

Sample label size:  torch.Size([50, 15])


In [7]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [8]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        embeds = self.embedding(x)
#         print(embeds.size())

        lstm_out, hidden = self.lstm(embeds, hidden)
#         print(lstm_out.size())


        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
#         print(lstm_out.size())
        
        
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
#         print(out.size())
        
        
        # sigmoid function
        sig_out = self.sig(out)
#         print(sig_out.size())
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size,200, -1)
#         print(sig_out.size())
        
        sig_out = sig_out[:, -1] # get last batch of labels
#         print(sig_out.size())
        
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [9]:
# Instantiate the model w/ hyperparams
vocab_size = 54898 + 1 # +1 for the 0 padding + our word tokens
output_size = 15
embedding_dim = 50
hidden_dim = 128
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(54899, 50)
  (lstm): LSTM(50, 128, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=128, out_features=15, bias=True)
  (sig): Sigmoid()
)


In [10]:
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [11]:
# training params

epochs = 15

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        inputs = inputs.type(torch.cuda.LongTensor)
        output, h = net(inputs, h)
#         print(output.size(),labels.size())
        
        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()
#                 print(val_h[0].size(),val_h[1].size())
                inputs = inputs.type(torch.cuda.LongTensor)
                output, val_h = net(inputs, val_h)
                
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/15... Step: 100... Loss: 0.376973... Val Loss: 0.357071
Epoch: 1/15... Step: 200... Loss: 0.348116... Val Loss: 0.333703
Epoch: 1/15... Step: 300... Loss: 0.286976... Val Loss: 0.311330
Epoch: 1/15... Step: 400... Loss: 0.286610... Val Loss: 0.297503
Epoch: 1/15... Step: 500... Loss: 0.267575... Val Loss: 0.290809
Epoch: 1/15... Step: 600... Loss: 0.279701... Val Loss: 0.276615
Epoch: 1/15... Step: 700... Loss: 0.270769... Val Loss: 0.275726
Epoch: 2/15... Step: 800... Loss: 0.272678... Val Loss: 0.258964
Epoch: 2/15... Step: 900... Loss: 0.244090... Val Loss: 0.244603
Epoch: 2/15... Step: 1000... Loss: 0.241695... Val Loss: 0.235284
Epoch: 2/15... Step: 1100... Loss: 0.232106... Val Loss: 0.225948
Epoch: 2/15... Step: 1200... Loss: 0.195684... Val Loss: 0.215620
Epoch: 2/15... Step: 1300... Loss: 0.213123... Val Loss: 0.216200
Epoch: 2/15... Step: 1400... Loss: 0.179618... Val Loss: 0.201092
Epoch: 3/15... Step: 1500... Loss: 0.178423... Val Loss: 0.193319
Epoch: 3/15... Step

In [12]:
test_x = np.load('test_x.npy')

In [13]:
test_x.shape

(10393, 200)

In [14]:
feature_tensor = torch.from_numpy(test_x)
print(feature_tensor.size())

torch.Size([10393, 200])


In [15]:
predictions = []
def predict(net, feature_tensor):
    
    net.eval()
    
    batch_size = 1
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
    
    if(train_on_gpu):
        feature_tensor = feature_tensor.cuda()
        
    feature_tensor = feature_tensor.type(torch.cuda.LongTensor)
    # get the output from the model

    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())
    return pred      

In [16]:
i = 0
for feature in feature_tensor:
    feature = feature.view(1,-1)
    pred = predict(net,feature)
    
    pred = pred.cpu().detach().numpy()
    
    predictions.append(pred)
    
    i += 1
    if i%500 == 0:
        print(i)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000


In [17]:
sub = pd.DataFrame(predictions, columns=['label_Assembly Required', 'label_Color',
       'label_Commercial / Residential', 'label_ENERGY STAR Certified',
       'label_Features', 'label_Finish', 'label_Flooring Product Type',
       'label_Hardware Included', 'label_Included', 'label_Indoor/Outdoor',
       'label_Package Quantity', 'label_Shape', 'label_Tools Product Type',
       'label_Voltage (volts)', 'label_Wattage (watts)'])

In [18]:
sub

,label_Assembly Required,label_Color,label_Commercial / Residential,label_ENERGY STAR Certified,label_Features,label_Finish,label_Flooring Product Type,label_Hardware Included,label_Included,label_Indoor/Outdoor,label_Package Quantity,label_Shape,label_Tools Product Type,label_Voltage (volts),label_Wattage (watts)
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
